### Image preprocessing for DJI Mavic 2 Pro and Phantom 4 Multispectral (P4M) data
##### _Last update: 27 July 2022_
##### _Authors: Andrii Zaiats, Valorie Marie_
##### Purpose: This file includes image preprocessing steps that can be useful for SfM and multispectral workflows. The following steps are included:

RGB images:  

1. Converting proprietary image formats (e.g., Sony .ARW) to .dng format.
2. Converting .dng files to .jpg
3. Applying CLAHE (ontrast-limited adaptive historgram equalization) image corrections
4. Copying metadata from .dng to the newly created .jpg files  

Multispectral images:  

5. DJI Phantom 4 Multispectral images normalization.

Required software:
* <a href="https://helpx.adobe.com/camera-raw/using/adobe-dng-converter.html">Adobe DNG Converter<a>
* <a href="https://imagemagick.org/">ImageMagick<a>
* <a href="https://exiftool.org/">ExifTool<a>
* <a href="https://github.com/micasense/imageprocessing">MicaSense GitHub Image processing repository (_see MicaSense Image Processing Setup.ipynb_)<a>

####  1. Converting proprietary image formats (e.g., Sony .ARW) to .dng format.
This steps relies on Adobe DNG Converter software (available freely).  

Instructions:  
* Select intput/output locations: input directory should contain .ARW files, output directory is where the new .dng files will apprear
* the rest of the default settings should be fine
* click Exit and verify that the new .dng files are where they should be

#### 2. Converting .dng files to .jpg  

First, specify the path where the .dng files are located (_e.g., `path = 'D:/folder1/folder2/'`_)

In [1]:
path = 'D:/Andrii/20220603_InitialPoint/mavic_rgb_images_tif/'

Next, run ImageMagick command using python interface. (There exists a designated python package for ImageMagick but currently it is not implmenented).  
_Input:_ a path to the folder with .dng images (specified above).  
_Output:_ a copy of .dng images in .jpg format in the same directory where the .dng files are. That is okay, after modifying the images we can transfer the files over to a designated, final location.

In [2]:
cmd = 'magick mogrify -format jpg -quality 100 ' + path + '*.DNG'
os.system(cmd)

0

#### 3. Applying CLAHE (contrast-limited adaptive historgram equalization) image corrections
Thic correction appears to improve the contrast while mitigating shadows in the images. Not sure what effect this may have for the ODM reconstruction process, but since .jpg file format is mostly used for SfM, changing color properties of the images may help.  

_Note: Originally, this correction required adjusting the colours in Lightroom before running the CLAHE correction (not implemented here, but maybe could be done with ImageMagick). If implemented, the colour corrections should be Shadows & Blacks +100 and Highlights & Whites -100_

In [3]:
import cv2 as cv
from os import listdir
from os.path import join

files = [s for s in listdir(path) if s.endswith('.jpg')]

# provide path to the images
for n in range(0, len(files)):
    
    fimg = join(path, files[n])
    img = cv.imread(fimg)

    img_yuv = cv.cvtColor(img, cv.COLOR_BGR2YUV)
    clahe = cv.createCLAHE(clipLimit = 2, tileGridSize = (8,8))

    img_yuv[:,:,0] = clahe.apply(img_yuv[:,:,0])
    img_output = cv.cvtColor(img_yuv, cv.COLOR_YUV2BGR)

    output_image = join(path, files[n])
    cv.imwrite(output_image, img_output)


#### 4. Copying metadata from .dng to .jpg files
After steps 2 and 3, we should have a collection of .jpg files (CLAHE corrected or not) in the same folder where the original .dng files are. In this step, we use `exiftool` package to copy the metadata from the original .dng files to the newly created (and corrected) images. After copying over the metadata, this cell also moves the .jpg files to another, user specified, directory.

In [ ]:
import shutil 

cmd1 = 'exiftool -tagsfromfile %d%f.dng -all:all -overwrite_original -ext jpg ' + path 
os.system(cmd1)

for f in files:
    shutil.move(join(path, f), join(path + '/jpg'))

##### 5. Multispectral radiometric calibration
The following code is relevant to the processing P4M multispectral images only. Specifically, it implements radiometric calibration  using the DLS (on-board Downdwelling Light Sensor on Phantom 4 Multispectral). The code implements a package written specifically for this task (https://github.com/gdslab/p4m), which, in turn, heavily borrows from the MicaSense tutorials and source code.  

_Note: the default exiftool package does not work. The solution is to use a previous version of the package (`pip install pyexiftool==0.4.13` from https://github.com/micasense/imageprocessing/issues/177 )._  

_Note: to use MicaSense source code from the GitHub repo the repository needs to be activated from the specific Anaconda environment each time the code is used. If using VSCode Editor, in the top-right corner click on the current kernel and make sure to select 'micasense(Python 3.7.23)'._  


In [10]:
os.chdir('C:/Users/caugh/p4m')
os.system('conda activate P4M')
os.system('python raw2ref.py D:/Valorie/2022_field_campaign/NorthHam/multispec_images/ D:/Valorie/2022_field_campaign/NorthHam/multispec_images_refl/')

0